### Cell: Setup and Environment sanity check

**Goal**
- Ensure the runtime has the required libraries and GPU access via AWS Elastic Compute Cloud (EC2).

**What I did**
- For this project, I experimented with the AWS EC2 computing service. Specifically, I purchased an allocation for a G5.xlarge instance, which provide powerful GPU computing power.

**Output / Key result**
If CUDA is available, we should see `Cuda available: True` and a GPU name (e.g., an NVIDIA A10G on the g5.xlarge instance).

In [ ]:
import os, sys
import torch
from ultralytics import YOLO
from pathlib import Path


print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("No GPU detected")


## Test 00: Validation-only baseline (no fine-tuning)

**Goal**
Establish a baseline performance using a pre-trained YOLOv8n model without training on SODA-D dataset.

**What this test measures**
- How well off-the-shelf weights transfer to SODA-D dataset.
- A reference point for whether or not fine-tuning helps.

**Expected outcome**
This baseline is usually low for domain-specific small-object datasets, but it is useful nonetheless.

#### Analysis

**Quantitative**

//TODO: add
**Takeaways**

In [ ]:
## Code for test 00
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
print("running test 00")

metrics = model.val(
    data="/home/ubuntu/project/ElectricalEngineering428Project/configs/soda.yaml",
    imgsz=640,
    device=0,
)

print(metrics)

### Test 01: Training run (640×640 baseline)

**Goal**
Fine-tune YOLOv8n on SODA-D at 640×640 to establish a trained baseline.

**Why this matters**
This is the core comparison point for later experiments. If the pipeline is
correct, this run should outperform the validation-only baseline (Test 00).

**What to look for in outputs**
- A `runs/detect/train*/results.csv` file with improving mAP over epochs
- `weights/best.pt` saved for later validation/qualitative examples

**Brief analysis (fill after running)**
- Best epoch (by mAP50–95): ___
- Best mAP50–95 / mAP50: ___ / ___
- Precision / Recall: ___ / ___

**Next**
Run the higher-resolution experiment (Test 02).



Test 01

In [ ]:
## Code for test 01
import os, sys

from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))
print("running test 01")
print("Project root: ", PROJECT_ROOT)

from src.train_yolo import train_baseline

train_baseline(epochs=20,
               imgsz=640,
               batch=8,
)

## Test 02: Fine-tune YOLOv8n at 960x960 (higher resolution)

**Goal**
Test whether higher input resolution improves small-object detection by preserving more pixel details.

**What changed vs Test 01**
- `imgsz` increased from 640 -> 960
- Other parameters kept contant (`epochs=20`, `batch=8`)

**What to look for**
Small objects occupy very few pixels at 640x640, so increasing resolution should increase recall and improve overall AP.

#### Analysis

**Quantitative**

**Takeaways**

In [ ]:
## Code for test 02
import os, sys

from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))
print("running test 01")
print("Project root: ", PROJECT_ROOT)

from src.train_yolo import train_baseline

train_baseline(epochs=20,
               imgsz=960,
               batch=8,
)


### Test 03: Motivation for hyperparameter update (second 640×640 run)

**Goal**
Run another 640×640 training experiment, but with targeted augmentation /
training adjustments, to see whether we can improve small-object performance
without increasing resolution.

**What changed (vs Test 01)**
We enable multi-scale training and adjust scale + HSV augmentation:

- `multi_scale=True`: train on varying input sizes to improve scale robustness
- `scale=0.9`: allow stronger random resizing; can help generalization
- HSV jitter (`hsv_h=0.02, hsv_s=0.7, hsv_v=0.4`): improves robustness to lighting

**Hypothesis**
If performance improves at 640×640, then augmentation/training strategy is a
viable alternative to increasing resolution. If not, resolution is likely the
dominant factor for this dataset.

**Brief analysis (fill after running)**
- Best epoch (by mAP50–95): ___
- Best mAP50–95 / mAP50: ___ / ___
- Precision / Recall: ___ / ___

**Next**
Compile results into a single summary table + export artifacts (Test 04).


Test 03

In [ ]:
## Code for test 03
import os, sys

from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))
print("running test 01")
print("Project root: ", PROJECT_ROOT)

from src.train_yolo import train_baseline

train_baseline(epochs=20,
               imgsz=640,
               batch=8,
)

Test 04

In [ ]:
## Code for test 04

TODO

- write analysis
- experiment with segmentation
- Re-write code for train_yolo to account for test 3 (this is fine for now)
